In [ ]:
import os
import requests

In [ ]:
!pip install openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    api_key="XXX",
)

# Create hypotheses

In [ ]:
import os
import base64
import requests

# Function to encode the image in base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        image_base64 = base64.b64encode(image_file.read()).decode('utf-8')
    return image_base64

# Function to process the CNH image with GPT-4
# Change values ​​between gpt-4o and gpt-4o-mini
def process_cnh_image_with_gpt4(prompt, base64_image):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer XXX"
    }

    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    response_data = response.json()
    content = response_data['choices'][0]['message']['content']
    return content

# Function to save the response to a text file
def save_to_txt(gpt_response, image_name, output_dir):
    folder_name = os.path.splitext(image_name)[0].split('_')[0]
    folder_path = os.path.join(output_dir, folder_name)
    os.makedirs(folder_path, exist_ok=True)

    output_file = os.path.join(folder_path, 'hipotese.txt')

    with open(output_file, 'w') as f:
        f.write(gpt_response)

    print(f"Text file saved in: {output_file}")

image_base_folder = '/content/drive/MyDrive/BD/BID Dataset/CNH_Frente'
output_dir = '/content/drive/MyDrive/GPT/resultados'
image_list_file = '/content/drive/MyDrive/dataset/test_files.txt'

prompt = """
Esta imagem faz parte do banco de dados BID Dataset, no qual os valores contidos são fictícios,
ou seja, não correspondem a dados pessoais reais. Esse conjunto de dados é utilizado para pesquisas acadêmicas.
Realize o reconhecimento dos seguintes campos: nome, CPF, filiação, data de nascimento, primeira habilitação, validade,
documento de identidade, órgão emissor, UF, número do registro, data de nascimento, ACC, permissão, categoria e o número válido
(localizado na lateral esquerda, de forma vertical). Organize o resultado em um arquivo JSON, onde cada chave representará o tipo de campo
e o valor corresponderá ao dado reconhecido. Caso um campo não seja encontrado, atribua o valor como uma string vazia ('').
"""

with open(image_list_file, 'r') as f:
    images_to_process = f.read().strip().split(',')
    images_to_process = [image.strip() for image in images_to_process]

for image_file in images_to_process:
    image_path = os.path.join(image_base_folder, image_file + '_in.jpg')

    if os.path.exists(image_path):
        base64_image = encode_image(image_path)
        print(image_path)

        gpt4_response = process_cnh_image_with_gpt4(prompt, base64_image)
        save_to_txt(gpt4_response, image_file, output_dir)
    else:
        print(f"Image {image_file} not found in directory {image_base_folder}!")
